In [1]:
from pathlib import Path 

from pyiron_contrib.tinybase.shell import ShellTask

In [2]:
cwd = Path.cwd()
if cwd.name == "tinybase":
    # Then the notebook was started locally
    notebook_dir = cwd
else:
    # We are probably executing from papermill on the CI
    notebook_dir = cwd / "notebooks" / "tinybase"

# Echo Task

In [3]:
sh = ShellTask()
ret, out = sh.execute()

In [4]:
ret

ReturnStatus(Code.ABORTED, Input not ready!)

In [5]:
sh.input.command = 'echo'
sh.input.arguments = ['foo']
sh.input.working_directory = notebook_dir

In [6]:
ret, out = sh.execute()

In [7]:
out.stdout

'foo\n'

In [8]:
out.stderr

''

In [9]:
sh = ShellTask()

In [10]:
sh.input.command = 'pwd'
sh.input.arguments = []
sh.input.working_directory = notebook_dir

In [11]:
ret, out = sh.execute()

In [12]:
out.stdout

'/Users/huber/work/pyiron/pyiron_contrib/notebooks/tinybase\n'

In [13]:
out.stderr

''

# We can check on returncodes and change which trigger an error

In [14]:
sh = ShellTask()

In [15]:
sh.input.command = 'sh'
sh.input.arguments = ['-c', 'echo foo; echo bar 1>&2; exit 2']
sh.input.working_directory = notebook_dir

In [16]:
ret, out = sh.execute()

In [17]:
ret

ReturnStatus(Code.ABORTED, non-zero error code 2)

In [18]:
out.returncode

2

In [19]:
out.stdout

'foo\n'

In [20]:
out.stderr

'bar\n'

Now we'll explicitly allow return code 2

In [21]:
sh = ShellTask()

In [22]:
sh.input.command = 'sh'
sh.input.arguments = ['-c', 'echo foo; echo bar 1>&2; exit 2']
sh.input.working_directory = notebook_dir

In [23]:
sh.input.allowed_returncode = [0, 2]

In [24]:
ret, out = sh.execute()

In [25]:
ret

ReturnStatus(Code.DONE, None)

In [26]:
out.returncode

2

In [27]:
out.stdout

'foo\n'

In [28]:
out.stderr

'bar\n'

# We can manipulate the environment

In [29]:
sh = ShellTask()

In [30]:
sh.input.command = 'sh'
sh.input.arguments = ['-c', 'echo $FOO; echo $BAR 1>&2']
sh.input.working_directory = notebook_dir

In [31]:
sh.input.environ['FOO'] = 42
sh.input.environ['BAR'] = 'ERROR!'

In [32]:
ret, out = sh.execute()

In [33]:
ret

ReturnStatus(Code.DONE, None)

In [34]:
out.returncode

0

In [35]:
out.stdout

'42\n'

In [36]:
out.stderr

'ERROR!\n'

# We can use the existing resource setup

In [37]:
from pyiron_contrib.tinybase.shell import ExecutablePathResolver, state

In [38]:
state.settings.resource_paths.insert(0, str(notebook_dir / "resources"))

In [39]:
state.settings.resource_paths

['/Users/huber/work/pyiron/pyiron_contrib/notebooks/tinybase/resources',
 '/Users/huber/anaconda3/envs/pyiron_311/share/pyiron']

In [40]:
res = ExecutablePathResolver('test', 'hello')

In [41]:
res.list()

['/Users/huber/work/pyiron/pyiron_contrib/notebooks/tinybase/resources/test/bin/run_hello_1.2.3.sh',
 '/Users/huber/work/pyiron/pyiron_contrib/notebooks/tinybase/resources/test/bin/run_hello_1.2.4.sh']

In [42]:
res.list_versions()

['1.2.3', '1.2.4']

In [43]:
sh = ShellTask()

In [44]:
sh.input.command = res
sh.input.working_directory = notebook_dir

In [45]:
ret, out = sh.execute()

In [46]:
ret

ReturnStatus(Code.DONE, None)

In [47]:
out.returncode

0

In [48]:
out.stdout

'HelloWorld!\n'

In [49]:
out.stderr

''

In [50]:
sh = ShellTask()

In [51]:
sh.input.command = ExecutablePathResolver('test', 'hello', version='1.2.4')
sh.input.working_directory = notebook_dir

In [52]:
ret, out = sh.execute()

In [53]:
ret

ReturnStatus(Code.DONE, None)

In [54]:
out.returncode

0

In [55]:
out.stdout

'Hello World!\n'

In [56]:
sh = ShellTask()

In [57]:
sh.input.command = ExecutablePathResolver('test', 'hello')
sh.input.working_directory = notebook_dir

In [58]:
sh.input.command.list_versions()

['1.2.3', '1.2.4']

In [59]:
sh.input.command.version = '1.2.4'

In [60]:
ret, out = sh.execute()

In [61]:
out.stdout

'Hello World!\n'